In [ ]:
# to check the data set in video form
from collections import Counter
from random import shuffle

train_data = np.load('training_data.npy')

for data in train_data:
    img = data[0]
    print(img.shape)
    choice = data[1]
    cv2.imshow('test', img)
    print(choice)
    
    if cv2.waitKey(25) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break

In [1]:
# -*- coding: utf-8 -*-
# Importing libraries
import numpy as np
import cv2
import time
from PIL import ImageGrab
import os
from PIL import Image

In [2]:
# Data check
# As the data is not balanced so,droping some frame from stay option. 
import pandas as pd
from collections import Counter
train_data = np.load("training_data.npy")
df = pd.DataFrame(train_data)
print(df.head())
print(Counter(df[1].apply(str)))


                                                   0    1
0  [[255, 255, 255, 255, 255, 255, 255, 255, 255,...  [1]
1  [[255, 255, 255, 255, 255, 255, 255, 255, 255,...  [0]
2  [[255, 255, 255, 255, 255, 255, 255, 255, 255,...  [0]
3  [[255, 255, 255, 255, 255, 255, 255, 255, 255,...  [1]
4  [[255, 255, 255, 255, 255, 255, 255, 255, 255,...  [0]
Counter({'[0]': 3622, '[1]': 878})


In [4]:
# Balancing the data
from random import shuffle

jump = []
stay = []

shuffle(train_data)
for data in train_data:
    img = data[0]
    choice = data[1]
    
    if choice == [0]:
        stay.append([img, choice])
    elif choice == [1]:
        jump.append([img, choice])
    else:
        print("no matches!!!!!!")

        
stay = stay[:len(jump)]
jump = jump[:len(jump)]


final_data = stay + jump
shuffle(final_data)
print(len(final_data))

np.save('training_data_v2.npy', final_data)

2246


In [8]:
# alex net  to train the model

import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from tflearn.layers.normalization import local_response_normalization

curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Instructions for updating:
Colocations handled automatically by placer.


In [9]:
def alexnet(width, height, lr):
    network = input_data(shape=[None, width, height, 1], name='input')
    network = conv_2d(network, 96, 11, strides=4, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = local_response_normalization(network)
    network = conv_2d(network, 256, 5, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = local_response_normalization(network)
    network = conv_2d(network, 384, 3, activation='relu')
    network = conv_2d(network, 384, 3, activation='relu')
    network = conv_2d(network, 256, 3, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = local_response_normalization(network)
    network = fully_connected(network, 4096, activation='tanh')
    network = dropout(network, 0.5)
    network = fully_connected(network, 4096, activation='tanh')
    network = dropout(network, 0.5)
    network = fully_connected(network, 3, activation='softmax')
    network = regression(network, optimizer='momentum',
                         loss='categorical_crossentropy',
                         learning_rate=lr, name='targets')

    model = tflearn.DNN(network, checkpoint_path='model_alexnet',
                        max_checkpoints=1, tensorboard_verbose=2, tensorboard_dir='log')

    return model

In [10]:
# train_model.py

import numpy as np
#from alexnet import alexnet

WIDTH = 80
HEIGHT = 60
LR = 1e-3
EPOCHS = 8
MODEL_NAME = 'dino-run-{}-{}-{}-epochs.model'.format(LR, 'alexnetv2',EPOCHS)

model = alexnet(WIDTH, HEIGHT, LR)

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.cast instead.


In [11]:
# setup the training data
train_data = np.load('training_data_v2.npy')

train = train_data[:-500]
test = train_data[-500:]

X = np.array([i[0] for i in train]).reshape(-1,WIDTH,HEIGHT,1)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1,WIDTH,HEIGHT,1)
test_y = [i[1] for i in test]

In [17]:
print(Y)

[[0], [0], [1], [1], [0], [1], [1], [1], [0], [0], [0], [1], [1], [1], [0], [1], [1], [1], [1], [0], [0], [0], [0], [0], [1], [1], [0], [0], [1], [1], [0], [0], [1], [1], [1], [0], [1], [1], [1], [1], [1], [0], [0], [1], [0], [0], [1], [1], [0], [1], [1], [1], [1], [1], [1], [0], [1], [0], [0], [1], [0], [0], [1], [0], [1], [1], [0], [0], [1], [1], [1], [0], [0], [0], [1], [0], [1], [0], [1], [1], [1], [0], [1], [0], [0], [0], [1], [0], [0], [1], [1], [1], [0], [1], [1], [0], [1], [1], [0], [0], [1], [1], [1], [1], [0], [0], [0], [0], [1], [1], [0], [0], [0], [0], [1], [0], [1], [0], [1], [1], [1], [1], [0], [0], [1], [0], [1], [1], [1], [1], [0], [1], [0], [0], [1], [1], [0], [1], [1], [0], [0], [0], [0], [1], [0], [0], [1], [1], [1], [0], [0], [1], [0], [1], [0], [1], [0], [0], [0], [1], [0], [1], [1], [0], [0], [1], [0], [1], [0], [0], [1], [1], [0], [0], [0], [1], [0], [0], [0], [0], [0], [0], [1], [0], [0], [1], [0], [1], [0], [0], [1], [0], [1], [0], [0], [1], [1], [1], [1], [1],

In [12]:
model.fit({'input': X}, {'targets': Y}, n_epoch=EPOCHS, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

# tensorboard --logdir=foo:C:/Users/H/Desktop/ai-gaming/log

model.save(MODEL_NAME)

---------------------------------
Run id: dino-run-0.001-alexnetv2-8-epochs.model
Log directory: log/
INFO:tensorflow:Summary name Accuracy/ (raw) is illegal; using Accuracy/__raw_ instead.
---------------------------------
Training samples: 1746
Validation samples: 500
--


ValueError: Cannot feed value of shape (64, 1) for Tensor 'targets/Y:0', which has shape '(?, 3)'